In [3]:
import numpy as np
import torch
#import models.u_transformers_2D as ut2d
#import models.mymod.PatchUNet as pun
import matplotlib.pyplot as plt

from alltrain.DiceScore import DiceLoss

### Dice Loss

In [4]:
c = 2
hw = 128
bs = 4
y = 0*torch.from_numpy(np.random.randint(0,c,(bs,hw,hw))).float().cuda()
x = 0*torch.from_numpy(np.random.rand(bs,c,hw,hw)).float().cuda()
x[:,0,...] += 1



dice = DiceLoss(c)

print(x.shape)
print(y.shape)
l = dice(x,y)
print(l.item())

torch.Size([4, 2, 128, 128])
torch.Size([4, 128, 128])
num : tensor([47910.6562,     0.0000], device='cuda:0')
den1 : tensor([35025.4961,  4740.1855], device='cuda:0')
den2 : tensor([65536.,     0.], device='cuda:0')
dice : tensor([9.5286e-01, 2.1096e-09], device='cuda:0')
0.9528627991676331
0.47643139958381653
0.5235686302185059


In [2]:
eps = 1e-5
ret = 1-(2*bs*hw**2 + eps)/(bs*hw**2 + bs*hw**2 + eps)
print(ret)


0.0


In [3]:
bs*hw**2

65536

In [8]:
num = torch.sum(x, dim=3)
num = torch.sum(num, dim=2)
num = torch.sum(num, dim=0)
print(num.shape)

torch.Size([2])


In [6]:
s = torch.nn.functional.softmax(x, dim = 1)
s.shape

torch.Size([4, 2, 128, 128])

In [7]:
np.unique(s.cpu().numpy())

array([0.26894143, 0.7310586 ], dtype=float32)

## patched unet

In [6]:
mod = pun.Patched3DUNet(patch_size=(16,16,16), filters=(2,4,8,16,32), n_classes=2, in_channels=1)

bs, c, x, y, z = 1,1,16*12,16*12,15*7
inp = torch.from_numpy(np.random.rand(bs,c,x,y,z)).float()

In [ ]:
mod.eval()
out, count = mod(inp)

In [ ]:
print(out.shape)
print(count.shape)

In [ ]:
nz = 150 - 20
plt.imshow(count[0,0,0,:5,:])
plt.show()

In [6]:
a,b = np.unique((count[0,0,0,0,:]), return_counts=True)

In [7]:
print(a)
print(b)

[1. 2.]
[140  10]


In [1]:
15*7

105

In [ ]:
105%

In [8]:
count[0,0,0,0,:]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.])

In [ ]:
exit(0)

## transformers

In [ ]:
w, h = 256,256
trans = ut2d.u_transformers_2D(filters = [4,8,16,32,64], trans_shape=(64,16,16), n_classes=2,in_channels=1).cuda()
x = torch.from_numpy(np.random.rand(2,1,w, h)).float().cuda()
y = trans(x)

In [ ]:
32*16

In [ ]:
(1024*((512/(2**4)))**2)/(1000**2)

In [ ]:
(512*(512/(2**3))**2)/(1000**2)

In [ ]:
4398046511104/(1000**3)

In [ ]:
[i for i in [[1,2,3],[6,5,4],[7,8,9]] ]

--------------------

In [ ]:
def softDice(pred, target, smoothing=1, nonSquared=False):
    intersection = (pred * target).sum(dim=(1, 2, 3))
    if nonSquared:
        union = (pred).sum() + (target).sum()
    else:
        union = (pred * pred).sum(dim=(1, 2, 3)) + (target * target).sum(dim=(1, 2, 3))
    dice = (2 * intersection + smoothing) / (union + smoothing)

    #fix nans
    dice[dice != dice] = dice.new_tensor([1.0])

    return dice.mean()

def dice(pred, target):
    predBin = (pred > 0.5).float()
    return softDice(predBin, target, 0, True).item()

In [ ]:
y = torch.from_numpy(np.ones((1,1,80,80,32)))
p = torch.from_numpy(np.ones((1,1,80,80,32)))

In [ ]:
dice(y,p)

### Reversible

In [ ]:
def count_parameters(model): 
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
import expconfigs.multi_atlas_revunet_01_v2 as cfg_rev
excfg_rev = cfg_rev.ExpConfig()
nr = (count_parameters(excfg_rev.net))
print(excfg_rev.experiment_name)

### Classic

In [ ]:
import expconfigs.multi_atlas_unet_res01_v2 as cfg_cla
excfg_cla = cfg_cla.ExpConfig()
nc = (count_parameters(excfg_cla.net))

### Compare

In [ ]:
print("rev :", nr)
print("cla :", nc)
print("nc/nr :", nc/nr)
print("nr/nc :", nr/nc)
print('min :', min(nc/nr, nr/nc))

In [ ]:
print("rev :", nr)
print("cla :", nc)
print("nc/nr :", nc/nr)
print("nr/nc :", nr/nc)
print('min :', min(nc/nr, nr/nc))

In [ ]:
print("rev :", nr)
print("cla :", nc)
print("nc/nr :", nc/nr)
print("nr/nc :", nr/nc)
print('min :', min(nc/nr, nr/nc))